In [96]:
import numpy as np
import json
import pandas as pd

In [8]:
dataDir = "./data/openface"

In [52]:
def readOpenFaceFile(filename) -> [dict]:
  lines = []
  with open(f"{dataDir}/{filename}", "r") as f:
      for l in f:
        detectionStr = l.split(",", maxsplit=1)[1].rstrip().strip("\"")
        detectionStr = detectionStr.replace("\'", "\"")
        lines.append(json.loads(detectionStr))
  return lines

In [108]:
def getNumOfNoDetections(frames: [dict]) -> int:
  return len([f for f in frames if f["intensity"] == "-1" or f["intensity"]["AU01"] == "-"])

In [63]:
def getNumOfZeroIntensity(frames: [dict]) -> int:
  return len([f for f in frames if f["intensity"] != "-1" and f["intensity"]["AU06"] == 0 and f["intensity"]["AU12"] == 0])

In [102]:
videos = ["bright", "dark", "moving_background", "outside"]
bitrates = ["750", "1500"]
methods = ["none", "face", "fes"]

In [109]:
valuesDetect = []

for v in videos:
    for b in bitrates:
        for m in methods:
            frames = readOpenFaceFile(f"{v}_{b}_{m}.csv")
            noDetecs = getNumOfNoDetections(frames)
            zeroItensity = getNumOfZeroIntensity(frames)
            ratio = noDetecs/len(frames)
            valuesDetect.append(ratio)
            # print(f"{v}_{b}_{m}: {noDetecs}/{len(frames)} = {ratio:.3f}")

valuesDetect = np.array(valuesDetect).reshape(len(videos),len(bitrates),len(methods))

print("Avg no detections per method")
a = valuesDetect.reshape(len(videos)*len(bitrates),len(methods))
avgs = np.mean(a, axis=0)
print(pd.DataFrame(avgs, index=methods))

print("Avg no detections per method and bitrate")
avgs = np.mean(valuesDetect, axis=0)
print(pd.DataFrame(avgs, index=bitrates, columns=methods))

Avg no detections per method
             0
none  0.225697
face  0.223254
fes   0.222129
Avg no detections per method and bitrate
          none      face       fes
750   0.212696  0.197885  0.204870
1500  0.238698  0.248624  0.239388
